#### SQLite port_lite database: orders table
#### csv files: orders, ORDERS-PLACED

In [1]:
import calendar
import pandas as pd
import panel as pn
import seaborn as sns
from datetime import date, timedelta
from sqlalchemy import create_engine
from itables import init_notebook_mode, show
init_notebook_mode(all_interactive=False)

pn.extension('tabulator')

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"

def categorise(row):  
    if row['trade'] == 'B':
        return 'Buy'
    else:
        return 'Sell'
    
today = date.today()
today

<IPython.core.display.Javascript object>

datetime.date(2022, 5, 5)

### Begin of Tables in the process

In [2]:
cols = 'trade name qty price active reason market xdate'.split()
colt = 'trans name qty target active spd current change percent reason market xdate'.split()

In [3]:
format_dict = {
    'qty':'{:,}',
    'price':'{:.2f}','target':'{:.2f}','current':'{:.2f}','change':'{:.2f}','diff':'{:.2f}',
    'amount':'{:,.2f}','sell_amt':'{:,.2f}'
}

In [4]:
pd.read_sql_query('SELECT * FROM orders ORDER BY id DESC LIMIT 1', conlite).style.format(format_dict)

,id,trade,name,qty,price,active,reason,market,xdate
0,19,B,DCC,"30,000",2.98,2,HD,SET,2022-05-13


In [5]:
tmp = pd.read_csv('../data/orders-log.csv')
tmp['trans'] = tmp.apply(lambda row: categorise(row), axis=1)
tmp['amount'] = tmp.qty * tmp.target
tmp[tmp.active == 2].style.background_gradient(cmap = 'Blues')

,trade,name,spd,reason,market,qty,target,current,change,percent,active,xdate,trans,amount
1,B,DCC,0,HD,SET,30000,2.980000,2.980000,-0.080000,-2.61%,2,2022-05-13,Buy,89400.000000
5,S,DIF,0,CUT,SET,10000,14.200000,14.200000,0.100000,+0.71%,2,2022-02-02,Sell,142000.000000
6,S,DOHOME,9,C20.25,SET100,2400,21.000000,20.100000,0.000000,0.00%,2,2022-02-02,Sell,50400.000000
10,S,MCS,3,C12.8,SET,25000,13.100000,12.800000,0.500000,+4.07%,2,2022-03-15,Sell,327500.000000
12,S,RATCH,7,DOS,SET50,2000,46.000000,44.250000,0.250000,+0.57%,2,2022-03-17,Sell,92000.000000


In [6]:
tmp.head().sort_values('spd',ascending=False).style.hide_index()

trade,name,spd,reason,market,qty,target,current,change,percent,active,xdate,trans,amount
B,DCC,0,HD,SET,30000,2.980000,2.980000,-0.080000,-2.61%,2,2022-05-13,Buy,89400.000000
B,BCH,-2,RD03pct,SET100,6000,20.900000,21.100000,-0.400000,-1.86%,1,2022-05-10,Buy,125400.000000
B,JASIF,-2,ROUND,SET,10000,10.800000,11.000000,0.000000,0.00%,1,2022-03-03,Buy,108000.000000
B,RJH,-2,ROUND,SET,1500,33.000000,33.500000,0.000000,0.00%,1,2022-02-02,Buy,49500.000000
B,SENA,-7,RD05pct,SET,15000,4.280000,4.420000,-0.040000,-0.90%,1,2022-05-10,Buy,64200.000000


### End of Tables in the process

### Print to verify before upload file

In [7]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active, xdate
FROM orders
ORDER BY trade, name'''
orders = pd.read_sql(sql, conlite)

#df_tab = pn.widgets.Tabulator(orders, layout='fit_data', width=800)
df_tab = pn.widgets.Tabulator(orders)
df_tab

Tabulator(value=   trade    name  ...)

In [8]:
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

orders[cols].to_csv(output_file, header=True, index=False)
orders[cols].to_csv(data_file, header=True, index=False)
orders[cols].to_csv(box_file, header=True, index=False)
orders[cols].to_csv(one_file, header=True, index=False)

### End of transactional process

In [9]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active, xdate
FROM orders
WHERE active = 2
ORDER BY trade, name'''
df_act_2 = pd.read_sql(sql, conlite)
df_act_2.style.format(format_dict)

,trade,name,qty,price,amount,reason,market,active,xdate
0,B,DCC,"30,000",2.98,"89,400.00",HD,SET,2,2022-05-13
1,S,DIF,"10,000",14.20,"142,000.00",CUT,SET,2,2022-02-02
2,S,DOHOME,"2,400",21.00,"50,400.00",C20.25,SET100,2,2022-02-02
3,S,MCS,"5,000",13.50,"67,500.00",C12.8,SET,2,2022-03-15
4,S,RATCH,"2,000",46.00,"92,000.00",DOS,SET50,2,2022-03-17


In [10]:
orders_grp = df_act_2.groupby(['trade'])
orders_grp['amount'].sum()

trade
B     89400.0
S    351900.0
Name: amount, dtype: float64

In [11]:
mask = df_act_2.trade == 'B'
buy_stocks = df_act_2[mask]
buy_stocks.style.background_gradient(cmap = 'Blues').format(format_dict)

,trade,name,qty,price,amount,reason,market,active,xdate
0,B,DCC,"30,000",2.98,"89,400.00",HD,SET,2,2022-05-13


In [12]:
cash = 183_000
buy_amt = buy_stocks.amount.sum()
balance = cash - buy_amt 
cash, buy_amt, balance

(183000, 89400.0, 93600.0)

In [60]:
mask = df_act_2.trade == 'S'
sell_stocks = df_act_2[mask]
sell_stocks.style.format(format_dict)

,trade,name,qty,price,amount,reason,market,active,xdate
1,S,DIF,"10,000",14.20,"142,000.00",CUT,SET,2,2022-02-02
2,S,DOHOME,"2,400",21.00,"50,400.00",C20.25,SET100,2,2022-02-02
3,S,MCS,"25,000",13.10,"327,500.00",C12.8,SET,2,2022-03-15
4,S,RATCH,"2,000",46.00,"92,000.00",DOS,SET50,2,2022-03-17
5,S,SPALI,"3,000",22.40,"67,200.00",C22.0,SET100,2,2022-02-02


In [61]:
sell_amt = round(sell_stocks.amount.sum(),2)
balance = cash + sell_amt 
(cash, sell_amt, balance)

(231000, 679100.0, 910100.0)

### After call ord-log (must call everytime that orders change)

In [62]:
df = pd.read_csv('../data/orders-log.csv')
df['trans'] = df.apply(lambda row: categorise(row), axis=1)
df['amount'] = df.qty * df.target

In [64]:
show(df[['trans','name','spd','reason','qty','target','current','active']])

trans,name,spd,reason,qty,target,current,active


In [65]:
mb = (df.trans == 'Buy') & (df.spd >= -3)
df[mb].sort_values(['name'],ascending=[True])

,trade,name,spd,reason,market,qty,target,current,change,percent,active,xdate,trans,amount
0,B,BCH,-2,RD03pct,SET100,6000,20.90,21.10,-0.40,-1.86%,1,2022-05-10,Buy,125400.0
1,B,DCC,0,HD,SET,30000,2.98,2.98,-0.08,-2.61%,2,2022-05-13,Buy,89400.0
2,B,JASIF,-2,ROUND,SET,10000,10.80,11.00,0.00,0.00%,1,2022-03-03,Buy,108000.0
3,B,RJH,-2,ROUND,SET,1500,33.00,33.50,0.00,0.00%,1,2022-02-02,Buy,49500.0


In [66]:
show(df[['trans','name','spd','reason','qty','target','current','percent','active']])

trans,name,spd,reason,qty,target,current,percent,active


In [67]:
ms = (df.trans == 'Sell') & (df.spd <= 4)
df[ms].sort_values(['name'],ascending=[True])

,trade,name,spd,reason,market,qty,target,current,change,percent,active,xdate,trans,amount
5,S,DIF,0,CUT,SET,10000,14.2,14.2,0.1,+0.71%,2,2022-02-02,Sell,142000.0
10,S,MCS,3,C12.8,SET,25000,13.1,12.8,0.5,+4.07%,2,2022-03-15,Sell,327500.0
14,S,SPALI,4,C22.0,SET100,3000,22.4,22.0,0.1,+0.46%,1,2022-02-02,Sell,67200.0


In [68]:
df[mb | ms].sort_values(['trade','name'],ascending=[True,True])

,trade,name,spd,reason,market,qty,target,current,change,percent,active,xdate,trans,amount
0,B,BCH,-2,RD03pct,SET100,6000,20.90,21.10,-0.40,-1.86%,1,2022-05-10,Buy,125400.0
1,B,DCC,0,HD,SET,30000,2.98,2.98,-0.08,-2.61%,2,2022-05-13,Buy,89400.0
2,B,JASIF,-2,ROUND,SET,10000,10.80,11.00,0.00,0.00%,1,2022-03-03,Buy,108000.0
3,B,RJH,-2,ROUND,SET,1500,33.00,33.50,0.00,0.00%,1,2022-02-02,Buy,49500.0
5,S,DIF,0,CUT,SET,10000,14.20,14.20,0.10,+0.71%,2,2022-02-02,Sell,142000.0
10,S,MCS,3,C12.8,SET,25000,13.10,12.80,0.50,+4.07%,2,2022-03-15,Sell,327500.0
14,S,SPALI,4,C22.0,SET100,3000,22.40,22.00,0.10,+0.46%,1,2022-02-02,Sell,67200.0


In [69]:
df[mb | ms].shape[0]

7

In [70]:
mask = (df.trade == 'B')
df[mask].nlargest(5, "amount")[["name","trans","amount","spd"]].style.format(format_dict)

,name,trans,amount,spd
0,BCH,Buy,"125,400.00",-2
2,JASIF,Buy,"108,000.00",-2
1,DCC,Buy,"89,400.00",0
4,SENA,Buy,"64,200.00",-7
3,RJH,Buy,"49,500.00",-2


### Select source of orders between these two

In [71]:
df_out = df[mb | ms][colt]
df_out

,trans,name,qty,target,active,spd,current,change,percent,reason,market,xdate
0,Buy,BCH,6000,20.90,1,-2,21.10,-0.40,-1.86%,RD03pct,SET100,2022-05-10
1,Buy,DCC,30000,2.98,2,0,2.98,-0.08,-2.61%,HD,SET,2022-05-13
2,Buy,JASIF,10000,10.80,1,-2,11.00,0.00,0.00%,ROUND,SET,2022-03-03
3,Buy,RJH,1500,33.00,1,-2,33.50,0.00,0.00%,ROUND,SET,2022-02-02
5,Sell,DIF,10000,14.20,2,0,14.20,0.10,+0.71%,CUT,SET,2022-02-02
10,Sell,MCS,25000,13.10,2,3,12.80,0.50,+4.07%,C12.8,SET,2022-03-15
14,Sell,SPALI,3000,22.40,1,4,22.00,0.10,+0.46%,C22.0,SET100,2022-02-02


In [72]:
df_out = df[df.active == 2][colt]
df_out

,trans,name,qty,target,active,spd,current,change,percent,reason,market,xdate
1,Buy,DCC,30000,2.98,2,0,2.98,-0.08,-2.61%,HD,SET,2022-05-13
5,Sell,DIF,10000,14.20,2,0,14.20,0.10,+0.71%,CUT,SET,2022-02-02
6,Sell,DOHOME,2400,21.00,2,9,20.10,0.00,0.00%,C20.25,SET100,2022-02-02
10,Sell,MCS,25000,13.10,2,3,12.80,0.50,+4.07%,C12.8,SET,2022-03-15
12,Sell,RATCH,2000,46.00,2,7,44.25,0.25,+0.57%,DOS,SET50,2022-03-17


In [73]:
file_name = 'ORDERS-PLACED.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(data_file, header=True, index=False)
df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(box_file, header=True, index=False)
df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(one_file, header=True, index=False)

### End of Output ORDERS-PLACED

In [29]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active, xdate
FROM orders
WHERE ACTIVE = 2
ORDER BY trade, name'''
orders = pd.read_sql(sql, conlite)
orders.style.format(format_dict)

,trade,name,qty,price,amount,reason,market,active,xdate
0,B,MAKRO,"1,500",37.00,"55,500.00",RD05pct,SET,2,2022-03-03
1,S,DIF,"10,000",14.20,"142,000.00",CUT,SET,2,2022-02-02
2,S,DOHOME,"2,400",21.00,"50,400.00",C20.25,SET100,2,2022-02-02
3,S,MCS,"25,000",13.10,"327,500.00",C12.8,SET,2,2022-03-15
4,S,RATCH,"2,000",46.00,"92,000.00",DOS,SET50,2,2022-03-17


### Evening Process

In [ ]:
new_active = 1
old_active = 2

In [ ]:
sql = """
UPDATE orders 
SET active = %s
WHERE active = %s
"""
sql = sql % (new_active, old_active)
rp = conlite.execute(sql)
rp.rowcount

In [ ]:
names = df_out['name']
in_p = ", ".join(map(lambda name: "'%s'" % name, names))
in_p

In [ ]:
sql = """
UPDATE orders 
SET active = 2
WHERE name IN (%s)
"""
sql = sql % (in_p)
rp = conlite.execute(sql)
rp.rowcount

In [ ]:
#To force orders.csv and orders table to be same copy
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

orders[cols].to_csv(output_file, header=True, index=False)
orders[cols].to_csv(data_file, header=True, index=False)
orders[cols].to_csv(box_file, header=True, index=False)
orders[cols].to_csv(one_file, header=True, index=False)

### End of Evening Process